In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import glob

# Get a list of all the file paths that ends with .txt from in specified directory
# #fmri_list = glob.glob("/content/drive/MyDrive/fmri_data1/*")
# fmri_list = glob.glob("/content/drive/MyDrive/fmri_all_sub01/sub-001/func/*T1w_desc-preproc_bold.nii.gz")\
fmri_list = glob.glob("/content/drive/MyDrive/fmri_all_sub01/sub-001/func/*2_desc-preproc_bold.nii.gz")
# sort and Print the file list
fmri_list.sort()
# fmri_list.reverse()
file_list = glob.glob("/content/drive/MyDrive/fmri_data1/sub-001/func/*.tsv")

# sort and Print the file list
file_list.sort()
list_genre = ["'metal'","'classical'","'rock'","'blues'","'country'","'disco'","'hiphop'","'reggae'","'pop'","'jazz'"]
arr_mapping  = [7,8,9,10,11,12,13,14,15,16,17,18,1,2,3,4,5,6]

In [ ]:
fmri_list

['/content/drive/MyDrive/fmri_all_sub01/sub-001/func/sub-001_task-Test_run-01_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz',
 '/content/drive/MyDrive/fmri_all_sub01/sub-001/func/sub-001_task-Test_run-02_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz',
 '/content/drive/MyDrive/fmri_all_sub01/sub-001/func/sub-001_task-Test_run-03_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz',
 '/content/drive/MyDrive/fmri_all_sub01/sub-001/func/sub-001_task-Test_run-04_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz',
 '/content/drive/MyDrive/fmri_all_sub01/sub-001/func/sub-001_task-Test_run-05_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz',
 '/content/drive/MyDrive/fmri_all_sub01/sub-001/func/sub-001_task-Test_run-06_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz',
 '/content/drive/MyDrive/fmri_all_sub01/sub-001/func/sub-001_task-Training_run-01_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz',
 '/content/drive/MyDrive/fmri_all_sub

In [ ]:
file_list

['/content/drive/MyDrive/fmri_data1/sub-001/func/sub-001_task-Test_run-01_events.tsv',
 '/content/drive/MyDrive/fmri_data1/sub-001/func/sub-001_task-Test_run-02_events.tsv',
 '/content/drive/MyDrive/fmri_data1/sub-001/func/sub-001_task-Test_run-03_events.tsv',
 '/content/drive/MyDrive/fmri_data1/sub-001/func/sub-001_task-Test_run-04_events.tsv',
 '/content/drive/MyDrive/fmri_data1/sub-001/func/sub-001_task-Test_run-05_events.tsv',
 '/content/drive/MyDrive/fmri_data1/sub-001/func/sub-001_task-Test_run-06_events.tsv',
 '/content/drive/MyDrive/fmri_data1/sub-001/func/sub-001_task-Training_run-01_events.tsv',
 '/content/drive/MyDrive/fmri_data1/sub-001/func/sub-001_task-Training_run-02_events.tsv',
 '/content/drive/MyDrive/fmri_data1/sub-001/func/sub-001_task-Training_run-03_events.tsv',
 '/content/drive/MyDrive/fmri_data1/sub-001/func/sub-001_task-Training_run-04_events.tsv',
 '/content/drive/MyDrive/fmri_data1/sub-001/func/sub-001_task-Training_run-05_events.tsv',
 '/content/drive/MyDriv

In [ ]:
import nibabel as nib
import pandas as pd
count = 0
for k in range (1):
    df = pd.read_csv(file_list[k], sep="\t")
    # load fmri
    fmri =nib.load(fmri_list[k])
    brain_run_data = fmri.get_fdata()
    # print(brain_run_data[40,40,40,4])

In [ ]:
import numpy as np
import pandas as pd
# X = np.zeros((738,96,96,68,10))
Y = np.zeros(738)

In [ ]:
brain_run_data.shape

(97, 115, 97, 410)

In [ ]:
def standardize_tensor(tensor):
    mean = np.mean(tensor)
    std_dev = np.std(tensor)
    return (tensor - mean) / std_dev
# Save the training files on disk
import nibabel as nib
import pandas as pd
count = 0
for k in range (18):
    df = pd.read_csv(file_list[k], sep="\t")

    # load fmri
    #fmri =nib.load(fmri_list[arr_mapping[k]])
    fmri =nib.load(fmri_list[k])
    brain_run_data = fmri.get_fdata()
    # load fmri
    for j in range(len(df)):
        # add Y
        row_info = df.iloc[j]
        Y[count] = list_genre.index(row_info["genre"])
        file_name = "Brain_"+str(count)+".npy"
        # save fmri
        # np.save(file_name,standardize_tensor(brain_run_data[:,:,:,j*10:10*j+10]))
        np.save(file_name,standardize_tensor(brain_run_data[18:39,67:81,22:39,j*10:10*j+10]))
        file_name = "Brain_"+str(count)+"y.npy"
        # save fmri
        np.save(file_name,Y[count])
        count+=1


In [ ]:
def get_index(n_sample):
  # Change to 0.7
  train_idx = np.random.choice(n_sample, size=int(n_sample*0.7), replace=False)
  val_and_test = [i for i in range(n_sample) if i not in train_idx]
  vt_size = len(val_and_test)
  val_index = np.random.choice(vt_size, size=int(vt_size*0.5), replace=False)
  val_idx = [val_and_test[i] for i in val_index]
  test_idx = [i for i in range(n_sample) if i not in train_idx and i not in val_idx]
  return train_idx, val_idx, test_idx

In [ ]:
Y

array([8., 2., 4., 3., 5., 0., 7., 1., 6., 9., 8., 2., 4., 3., 5., 0., 7.,
       1., 6., 9., 8., 2., 4., 3., 5., 0., 7., 1., 6., 9., 8., 2., 4., 3.,
       5., 0., 7., 1., 6., 9., 8., 0., 7., 2., 6., 8., 4., 9., 3., 1., 5.,
       0., 7., 2., 6., 8., 4., 9., 3., 1., 5., 0., 7., 2., 6., 8., 4., 9.,
       3., 1., 5., 0., 7., 2., 6., 8., 4., 9., 3., 1., 5., 0., 9., 6., 3.,
       2., 4., 8., 5., 1., 0., 7., 9., 6., 3., 2., 4., 8., 5., 1., 0., 7.,
       9., 6., 3., 2., 4., 8., 5., 1., 0., 7., 9., 6., 3., 2., 4., 8., 5.,
       1., 0., 7., 9., 9., 0., 4., 2., 7., 3., 6., 5., 1., 8., 9., 0., 4.,
       2., 7., 3., 6., 5., 1., 8., 9., 0., 4., 2., 7., 3., 6., 5., 1., 8.,
       9., 0., 4., 2., 7., 3., 6., 5., 1., 8., 9., 7., 8., 4., 1., 0., 5.,
       6., 2., 3., 9., 7., 8., 4., 1., 0., 5., 6., 2., 3., 9., 7., 8., 4.,
       1., 0., 5., 6., 2., 3., 9., 7., 8., 4., 1., 0., 5., 6., 2., 3., 9.,
       7., 7., 3., 8., 1., 5., 0., 9., 6., 4., 2., 7., 3., 8., 1., 5., 0.,
       9., 6., 4., 2., 7.

In [ ]:
from re import X

# Example usage

# Load your dataset here
# For this example, let's just create dummy datasets
num_samples = 738  # Number of samples in dummy dataset
num_classes = 10  # The number of classes you want to classify into

train_idx, val_idx, test_idx = get_index(num_samples)
# X_train = np.zeros((len(train_idx), 10, 96, 96, 68))
# y_train = np.zeros(( len(train_idx),))
# X_val = np.zeros((len(val_idx), 10, 96, 96, 68))
# y_val = np.zeros(( len(val_idx),))
# X_test = np.zeros((len(test_idx), 10, 96, 96, 68))
# y_test = np.zeros(( len(test_idx),))
# count = 0
# for i in train_idx:
#   X_train[count,:,:,:,:] = np.load("Brain_"+str(i)+".npy").transpose(3,0,1,2)
#   y_train[count] = Y[i]
#   count+=1
# count=0
# for i in val_idx:
#   X_val[count,:,:,:,:] = np.load("Brain_"+str(i)+".npy").transpose(3,0,1,2)
#   y_val[count] = Y[i]
#   count+=1
# count=0
# for i in test_idx:
#   X_test[count,:,:,:,:] = np.load("Brain_"+str(i)+".npy").transpose(3,0,1,2)
#   y_test[count] = Y[i]
#   count+=1
np.save("/content/drive/MyDrive/aime/CNN/train_idx.txt",train_idx)
np.save("/content/drive/MyDrive/aime/CNN/val_idx.txt",val_idx)
np.save("/content/drive/MyDrive/aime/CNN/test_idx.txt",test_idx)

In [ ]:
import tensorflow as tf
import numpy as np
import os
from pathlib import Path
batch_size =  4
# Function to load and preprocess your data
def load_and_preprocess_data(file_path):
    # Load your data from file (e.g., images, CSV, etc.)
    #print(file_path)
    file_path = bytes.decode(file_path.numpy())
    data =  np.load(file_path)
    # Preprocess the data (normalization, resizing, etc.)
    data = data.transpose(3,0,1,2)
    # print(data.shape)
    #slice data
    #data = data[4:,:,:,:]
    data = tf.convert_to_tensor(data)
    data = tf.expand_dims(data, axis=-1)
    file_arr = file_path.split(".")
    file_path_y = file_arr[0]+"y."+ file_arr[1]

    y_res = int(np.load(str(file_path_y)))
    one_hot = np.zeros(10)
    one_hot[y_res] = 1
    return data, tf.convert_to_tensor(one_hot)

# Create a dataset from the list of file paths
file_paths = ["Brain_"+str(i)+".npy" for i in train_idx]
# train_dataset = tf.data.Dataset.list_files('/content/bird.jpg')
dataset = tf.data.Dataset.list_files(file_paths)

# Map the preprocessing function across dataset elements
dataset = dataset.map(lambda x: tf.py_function(load_and_preprocess_data, [x], [tf.float64,tf.float64]),
                      num_parallel_calls=tf.data.AUTOTUNE)

# Shuffle, repeat, and batch the dataset
dataset = dataset.shuffle(buffer_size=16).repeat().batch(batch_size)

# Prefetch to improve performance
dataset = dataset.prefetch(tf.data.AUTOTUNE)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Input, BatchNormalization, ReLU, TimeDistributed, LSTM

def create_4d_cnn_with_spacetime_kernel(input_shape, num_classes):
    # Input shape should be (batch_size, time_steps, height, width, channels)
    inputs = Input(shape=input_shape)

    # Example spacetime convolutional layer (extend kernel over two frames)
    # Kernel size here is (3, 3, 3), which extends over temporal and spatial dimensions
    conv_3d_lay1 = Conv3D(filters=32, kernel_size=(5, 5, 5), strides=(1, 1, 1), padding='same')
    x = TimeDistributed(conv_3d_lay1)(inputs)
    batch1 = BatchNormalization()
    x =  TimeDistributed(batch1)(x)
    x = ReLU()(x)
    max_pool_lay1 = MaxPooling3D(pool_size=(2, 2, 2), strides=(1, 1, 1))
    x = TimeDistributed(max_pool_lay1)(x)

    conv_3d_lay2 = Conv3D(filters=64, kernel_size=(4, 4, 4), strides=(1, 1, 1), padding='same')
    x = TimeDistributed(conv_3d_lay2)(x)
    batch2 = BatchNormalization()
    x =  TimeDistributed(batch2)(x)
    x = ReLU()(x)
    max_pool_lay2 = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2))
    x = TimeDistributed(max_pool_lay2)(x)

    conv_3d_lay3 = Conv3D(filters=128, kernel_size=(3, 3, 3), strides=(2,2 ,2), padding='same')
    x = TimeDistributed(conv_3d_lay3)(x)
    batch3 = BatchNormalization()
    x =  TimeDistributed(batch3)(x)
    x = ReLU()(x)
    max_pool_lay3 = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2))
    x = TimeDistributed(max_pool_lay3)(x)

    # conv_3d_lay4 = Conv3D(filters=128, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same')
    # x = TimeDistributed(conv_3d_lay4)(x)
    # batch4 = BatchNormalization()
    # x =  TimeDistributed(batch4)(x)
    # x = ReLU()(x)
    # max_pool_lay4 = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2))
    # x = TimeDistributed(max_pool_lay4)(x)
    x = TimeDistributed(Flatten())(x)
    x = TimeDistributed(Dense(32))(x)
    x = LSTM(64,return_sequences=False)(x)


    # Flatten and add Dense layers at the end
    # # Add more layers according to your specific problem, for example:
    # x = Conv3D(filters=64, kernel_size=(3, 3, 3), strides=(1, 2, 2), padding='same')(x)
    # x = BatchNormalization()(x)
    # x = ReLU()(x)
    # x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2))(x)

    # # Flatten and add Dense layers at the end
    # x = Flatten()(x)
    # x = Dense(512)(x)
    # x = ReLU()(x)
    x = Dense(num_classes, activation='softmax')(x)

    # # Create model
    model = Model(inputs=inputs, outputs=x)

    return model

# Example input shape: 10 frames (time_steps), each frame with size 64x64, and 3 channels
input_shape = (10,21,14,17,1)
num_classes = 10  # Example number of classes for classification

model = create_4d_cnn_with_spacetime_kernel(input_shape, num_classes)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 10, 21, 14, 17, 1)   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_11                  │ (None, 10, 21, 14, 17, 32)  │           4,032 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_12                  │ (None, 10, 21, 14, 17, 32)  │             128 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_3 (ReLU)                       │ (None, 10, 21, 14, 17, 32)  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_13                  │ (None, 10, 20, 13, 16, 32)  │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_14                  │ (None, 10, 20, 13, 16, 64)  │         131,136 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_15                  │ (None, 10, 20, 13, 16, 64)  │             256 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_4 (ReLU)                       │ (None, 10, 20, 13, 16, 64)  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_16                  │ (None, 10, 10, 6, 8, 64)    │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_17                  │ (None, 10, 5, 3, 4, 128)    │         221,312 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_18                  │ (None, 10, 5, 3, 4, 128)    │             512 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_5 (ReLU)                       │ (None, 10, 5, 3, 4, 128)    │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_19                  │ (None, 10, 2, 1, 2, 128)    │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_20                  │ (None, 10, 512)             │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_21                  │ (None, 10, 32)              │          16,416 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 64)                  │          24,8

 Total params: 399,274 (1.52 MB)

 Trainable params: 398,826 (1.52 MB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
from tensorflow.keras.metrics import CategoricalAccuracy
# Example input shape: 10 frames (time_steps), each frame with size 64x64, and 3 channels
# input_shape = (10,96,96,68)

num_classes = 10  # Example number of classes for classification

model = create_4d_cnn_with_spacetime_kernel(input_shape, num_classes)
optimizer = tf.optimizers.Adam(learning_rate=1e-4)
loss_object = tf.losses.CategoricalCrossentropy(from_logits=False)
val_acc_metric = CategoricalAccuracy()

In [ ]:
@tf.function
def train_step(inputs, targets):
    with tf.GradientTape() as tape:

        predictions = model(inputs, training=True)
        #print(predictions)
        #print(targets)
        loss = loss_object(targets, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [ ]:
@tf.function
def perform_validation(model, x, y):
    val_logits = model(x, training=False)
    return val_logits

In [ ]:
epochs = 100
batch_size = 4
steps_per_epoch = len(train_idx) // batch_size
val_best = 0

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    loss_tot = 0
    # Iterate over the batches of the dataset.
    for step, (inputs, targets) in enumerate(dataset.take(steps_per_epoch)):
        loss = train_step(inputs, targets)
        loss_tot += loss.numpy()
        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss_tot/(step+1)}")

    # End of epoch actions...
    # Perform validation
    for idx in val_idx:
        x_batch_val = np.load("Brain_"+str(idx)+".npy")
        x_batch_val = x_batch_val.transpose(3,0,1,2)
        x_batch_val = x_batch_val.reshape(1,*x_batch_val.shape)
        # x_batch_val = x_batch_val.reshape(1,96,96,68,10)
        #x_batch_val = x_batch_val[4:,:,:,:].reshape(1,6,96,96,68)
        x_batch_val = tf.convert_to_tensor(x_batch_val)
        x_batch_val = tf.expand_dims(x_batch_val, axis=-1)
        y_batch_val = np.load("Brain_"+str(idx)+"y.npy")
        y_batch_val = int(y_batch_val)
        one_hot = np.zeros(10)
        one_hot[y_batch_val] = 1
        y_batch_val = tf.convert_to_tensor(one_hot)
        val_logits = perform_validation(model, x_batch_val, y_batch_val)
        # Update validation metrics
        val_acc_metric.update_state(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    if(val_acc > val_best):
        val_best = val_acc
        model.save("model_alt.h5")
    val_acc_metric.reset_state()
    print("Training Loss: %.4f" % (float(loss_tot)/len(train_idx),))
    print("Validation acc: %.4f" % (float(val_acc),))

Epoch 1/100
Step 0, Loss: 2.442028522491455
Step 100, Loss: 2.3537122436089093


Training Loss: 0.5881
Validation acc: 0.1171
Epoch 2/100
Step 0, Loss: 2.322967052459717
Step 100, Loss: 2.323551168536196


Training Loss: 0.5799
Validation acc: 0.1351
Epoch 3/100
Step 0, Loss: 2.3591854572296143
Step 100, Loss: 2.328776543683345
Training Loss: 0.5816
Validation acc: 0.1171
Epoch 4/100
Step 0, Loss: 2.2767462730407715
Step 100, Loss: 2.3165320996010657
Training Loss: 0.5785
Validation acc: 0.0631
Epoch 5/100
Step 0, Loss: 2.381899118423462
Step 100, Loss: 2.3147618841416766
Training Loss: 0.5794
Validation acc: 0.0811
Epoch 6/100
Step 0, Loss: 2.444847822189331
Step 100, Loss: 2.3108694506163645
Training Loss: 0.5782
Validation acc: 0.0901
Epoch 7/100
Step 0, Loss: 2.3461756706237793
Step 100, Loss: 2.3096656398017807
Training Loss: 0.5772
Validation acc: 0.0901
Epoch 8/100
Step 0, Loss: 2.306924343109131
Step 100, Loss: 2.307523715614092
Training Loss: 0.5773
Validation acc: 0.0901
Epoch 9/100
Step 0, Loss: 2.253807544708252
Step 100, Loss: 2.3079511788812015
Training Loss: 0.5763
Validation acc: 0.0721
Epoch 10/100
Step 0, Loss: 2.2553303241729736
Step 100, Loss: 2.3022945399331576
Train

KeyboardInterrupt: 

In [ ]:
21*14*17

4998

In [ ]:
# prompt: read in L_Heschl_gyrus_AAL_83.npy

import numpy as np

# Assuming the file is in the current working directory
brain_data_LH = np.load("L_Heschl_gyrus_AAL_83.npy")

# Now you can work with the loaded data, e.g., print its shape
print(brain_data_LH.shape)


(3, 594)


In [ ]:
# prompt: for each Brain_*.npy file zero out the voxels that does not belong in brain_data

import numpy as np

# Assuming brain_data is a boolean array indicating brain voxels
brain_mask = np.load("L_Heschl_gyrus_AAL_83.npy")
# brain_mask = np.load("/content/drive/MyDrive/aime/CNN/L_Heschl_gyrus_AAL_83.npy")


for j in range(num_samples):
  file_path = "Brain_"+str(j)+".npy"
  brain_data = np.load(file_path)
  new_brain_data = np.zeros(brain_data.shape)
  for i in range(brain_data_LH.shape[1]):
    new_brain_data[brain_data_LH[0,i]-18,brain_data_LH[1,i]-67,brain_data_LH[2,i]-22,:] = brain_data[brain_data_LH[0,i]-18,brain_data_LH[1,i]-67,brain_data_LH[2,i]-22,:]
  # Save the modified data back to the file
  np.save(file_path, new_brain_data)


In [ ]:
file_path = "Brain_1.npy"
brain_data = np.load(file_path)

In [ ]:
brain_data[0,0,0,:]

array([-2.3394529 , -2.34392719, -2.39638441, -2.31877858, -2.34871005,
       -2.37416723, -2.35580721, -2.37432152, -2.39607584, -2.33976147])

In [ ]:
new_brain_data = np.zeros(brain_data.shape)

In [ ]:
for i in range(brain_data_LH.shape[1]):
  new_brain_data[brain_data_LH[0,i]-18,brain_data_LH[1,i]-67,brain_data_LH[2,i]-22,:] = brain_data[brain_data_LH[0,i]-18,brain_data_LH[1,i]-67,brain_data_LH[2,i]-22,:]

In [ ]:
new_brain_data[:,:,10,1]

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.94930613,  0.96735759,
         0.98849476,  0.73716297,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.19731648,  0.38261389,
         0.40961393,  0.25100798,  0.12094494, -0.12699256, -1.15114261,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.13328781,  0.12341351,
         0.1849736 ,  0.28603089,  0.17772216,  0.46777972,  0.9068775 ,
         0.        ,  0.  

In [ ]:
num_samples

738

In [ ]:
epochs = 100
batch_size = 4
steps_per_epoch = len(train_idx) // batch_size
val_best = 0

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    loss_tot = 0
    # Iterate over the batches of the dataset.
    for step, (inputs, targets) in enumerate(dataset.take(steps_per_epoch)):
        loss = train_step(inputs, targets)
        loss_tot += loss.numpy()
        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss_tot/(step+1)}")

    # End of epoch actions...
    # Perform validation
    for idx in val_idx:
        x_batch_val = np.load("Brain_"+str(idx)+".npy")
        x_batch_val = x_batch_val.transpose(3,0,1,2)
        x_batch_val = x_batch_val.reshape(1,*x_batch_val.shape)
        # x_batch_val = x_batch_val.reshape(1,96,96,68,10)
        #x_batch_val = x_batch_val[4:,:,:,:].reshape(1,6,96,96,68)
        x_batch_val = tf.convert_to_tensor(x_batch_val)
        x_batch_val = tf.expand_dims(x_batch_val, axis=-1)
        y_batch_val = np.load("Brain_"+str(idx)+"y.npy")
        y_batch_val = int(y_batch_val)
        one_hot = np.zeros(10)
        one_hot[y_batch_val] = 1
        y_batch_val = tf.convert_to_tensor(one_hot)
        val_logits = perform_validation(model, x_batch_val, y_batch_val)
        # Update validation metrics
        val_acc_metric.update_state(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    if(val_acc > val_best):
        val_best = val_acc
        model.save("model_alt.h5")
    val_acc_metric.reset_state()
    print("Training Loss: %.4f" % (float(loss_tot)/len(train_idx),))
    print("Validation acc: %.4f" % (float(val_acc),))

Epoch 1/100
Step 0, Loss: 2.2542576789855957
Step 100, Loss: 2.327776037820495


Training Loss: 0.5810
Validation acc: 0.0631
Epoch 2/100
Step 0, Loss: 2.3220129013061523
Step 100, Loss: 2.3129209150182137


Training Loss: 0.5780
Validation acc: 0.0901
Epoch 3/100
Step 0, Loss: 2.2994589805603027
Step 100, Loss: 2.305446702654999
Training Loss: 0.5769
Validation acc: 0.0811
Epoch 4/100
Step 0, Loss: 2.2374677658081055
Step 100, Loss: 2.3056893820809847
Training Loss: 0.5766
Validation acc: 0.0450
Epoch 5/100
Step 0, Loss: 2.2740252017974854
Step 100, Loss: 2.3060994478735592
Training Loss: 0.5764
Validation acc: 0.0721
Epoch 6/100
Step 0, Loss: 2.3098349571228027
Step 100, Loss: 2.302339688386067
Training Loss: 0.5760
Validation acc: 0.0811
Epoch 7/100
Step 0, Loss: 2.3171310424804688
Step 100, Loss: 2.3043783773290047
Training Loss: 0.5760
Validation acc: 0.0721
Epoch 8/100
Step 0, Loss: 2.2666139602661133
Step 100, Loss: 2.3060050317556553
Training Loss: 0.5761
Validation acc: 0.0811
Epoch 9/100
Step 0, Loss: 2.325131416320801
Step 100, Loss: 2.303071673553769
Training Loss: 0.5758
Validation acc: 0.0360
Epoch 10/100
Step 0, Loss: 2.313225030899048
Step 100, Loss: 2.3013240323208346
Trai

KeyboardInterrupt: 

In [ ]:
# Build a simple LSTM

In [ ]:
# prompt: Build a simple LSTM model that takes in (594,10) input and output a 10 class classification for 10 temporal steps

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Build the LSTM model
model_l = Sequential()
model_l.add(LSTM(units=64, return_sequences=True, input_shape=(10, 594)))  # Adjust units as needed
model_l.add(LSTM(units=32, return_sequences=False))
model_l.add(Dense(units=10, activation='softmax'))

# Compile the model
model_l.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model_l.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_5 (LSTM)                        │ (None, 10, 64)              │         168,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 10)                  │             330 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 181,450 (708.79 KB)

 Trainable params: 181,450 (708.79 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
X_LSTM = np.zeros((738,10,594))
Y_LSTM = np.zeros(738)
for i in range(738):
  meta_data = np.load("Brain_"+str(i)+".npy")
  meta_data_y = np.load("Brain_"+str(i)+"y.npy")
  Y_LSTM[i] = meta_data_y
  for j in range(brain_data_LH.shape[1]):
    X_LSTM[i,:,j] = meta_data[brain_data_LH[0,j]-18,brain_data_LH[1,j]-67,brain_data_LH[2,j]-22,:]



In [ ]:
Y_LSTM

array([8., 2., 4., 3., 5., 0., 7., 1., 6., 9., 8., 2., 4., 3., 5., 0., 7.,
       1., 6., 9., 8., 2., 4., 3., 5., 0., 7., 1., 6., 9., 8., 2., 4., 3.,
       5., 0., 7., 1., 6., 9., 8., 0., 7., 2., 6., 8., 4., 9., 3., 1., 5.,
       0., 7., 2., 6., 8., 4., 9., 3., 1., 5., 0., 7., 2., 6., 8., 4., 9.,
       3., 1., 5., 0., 7., 2., 6., 8., 4., 9., 3., 1., 5., 0., 9., 6., 3.,
       2., 4., 8., 5., 1., 0., 7., 9., 6., 3., 2., 4., 8., 5., 1., 0., 7.,
       9., 6., 3., 2., 4., 8., 5., 1., 0., 7., 9., 6., 3., 2., 4., 8., 5.,
       1., 0., 7., 9., 9., 0., 4., 2., 7., 3., 6., 5., 1., 8., 9., 0., 4.,
       2., 7., 3., 6., 5., 1., 8., 9., 0., 4., 2., 7., 3., 6., 5., 1., 8.,
       9., 0., 4., 2., 7., 3., 6., 5., 1., 8., 9., 7., 8., 4., 1., 0., 5.,
       6., 2., 3., 9., 7., 8., 4., 1., 0., 5., 6., 2., 3., 9., 7., 8., 4.,
       1., 0., 5., 6., 2., 3., 9., 7., 8., 4., 1., 0., 5., 6., 2., 3., 9.,
       7., 7., 3., 8., 1., 5., 0., 9., 6., 4., 2., 7., 3., 8., 1., 5., 0.,
       9., 6., 4., 2., 7.

In [ ]:
# prompt: generate the training loop for X_LSTM and Y_LSTM for model_l

from tensorflow.keras.utils import to_categorical

# Convert Y_LSTM to one-hot encoding
Y_LSTM_cat = to_categorical(Y_LSTM, num_classes=10)

# Split data into training, validation, and test sets
X_train_l = X_LSTM[train_idx]
y_train_l = Y_LSTM_cat[train_idx]
X_val_l = X_LSTM[val_idx]
y_val_l = Y_LSTM_cat[val_idx]
X_test_l = X_LSTM[test_idx]
y_test_l = Y_LSTM_cat[test_idx]

# Train the LSTM model
epochs = 100
batch_size = 4
history = model_l.fit(
    X_train_l, y_train_l,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_val_l, y_val_l)
)


Epoch 1/100
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.1089 - loss: 2.3778 - val_accuracy: 0.0901 - val_loss: 2.3187
Epoch 2/100
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.1255 - loss: 2.3083 - val_accuracy: 0.0901 - val_loss: 2.3079
Epoch 3/100
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.1274 - loss: 2.3062 - val_accuracy: 0.1171 - val_loss: 2.3089
Epoch 4/100
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0678 - loss: 2.3113 - val_accuracy: 0.0450 - val_loss: 2.3181
Epoch 5/100
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0807 - loss: 2.3144 - val_accuracy: 0.0450 - val_loss: 2.3147
Epoch 6/100
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.1095 - loss: 2.3027 - val_accuracy: 0.0901 - val_loss: 2.3226
Epoch 7/100
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0883 - loss: 2.3092 - val_accuracy: 0.0631 - val_loss: 2.3173
Epoch 8/100
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0819 - loss: 2.3049 - val_accu

### R_Hesch_gyrus

In [ ]:
def standardize_tensor(tensor):
    mean = np.mean(tensor)
    std_dev = np.std(tensor)
    return (tensor - mean) / std_dev
# Save the training files on disk
import nibabel as nib
import pandas as pd
count = 0
for k in range (18):
    df = pd.read_csv(file_list[k], sep="\t")

    # load fmri
    #fmri =nib.load(fmri_list[arr_mapping[k]])
    fmri =nib.load(fmri_list[k])
    brain_run_data = fmri.get_fdata()
    # load fmri
    for j in range(len(df)):
        # add Y
        row_info = df.iloc[j]
        Y[count] = list_genre.index(row_info["genre"])
        file_name = "Brain_"+str(count)+".npy"
        # save fmri
        # np.save(file_name,standardize_tensor(brain_run_data[:,:,:,j*10:10*j+10]))
        np.save(file_name,standardize_tensor(brain_run_data[60:83,67:79,23:40,j*10:10*j+10]))
        file_name = "Brain_"+str(count)+"y.npy"
        # save fmri
        np.save(file_name,Y[count])
        count+=1


In [ ]:
def get_index(n_sample):
  # Change to 0.7
  train_idx = np.random.choice(n_sample, size=int(n_sample*0.7), replace=False)
  val_and_test = [i for i in range(n_sample) if i not in train_idx]
  vt_size = len(val_and_test)
  val_index = np.random.choice(vt_size, size=int(vt_size*0.5), replace=False)
  val_idx = [val_and_test[i] for i in val_index]
  test_idx = [i for i in range(n_sample) if i not in train_idx and i not in val_idx]
  return train_idx, val_idx, test_idx

In [ ]:
from re import X

# Example usage

# Load your dataset here
# For this example, let's just create dummy datasets
num_samples = 738  # Number of samples in dummy dataset
num_classes = 10  # The number of classes you want to classify into

train_idx, val_idx, test_idx = get_index(num_samples)
# X_train = np.zeros((len(train_idx), 10, 96, 96, 68))
# y_train = np.zeros(( len(train_idx),))
# X_val = np.zeros((len(val_idx), 10, 96, 96, 68))
# y_val = np.zeros(( len(val_idx),))
# X_test = np.zeros((len(test_idx), 10, 96, 96, 68))
# y_test = np.zeros(( len(test_idx),))
# count = 0
# for i in train_idx:
#   X_train[count,:,:,:,:] = np.load("Brain_"+str(i)+".npy").transpose(3,0,1,2)
#   y_train[count] = Y[i]
#   count+=1
# count=0
# for i in val_idx:
#   X_val[count,:,:,:,:] = np.load("Brain_"+str(i)+".npy").transpose(3,0,1,2)
#   y_val[count] = Y[i]
#   count+=1
# count=0
# for i in test_idx:
#   X_test[count,:,:,:,:] = np.load("Brain_"+str(i)+".npy").transpose(3,0,1,2)
#   y_test[count] = Y[i]
#   count+=1
np.save("/content/drive/MyDrive/aime/CNN/train_idx.txt",train_idx)
np.save("/content/drive/MyDrive/aime/CNN/val_idx.txt",val_idx)
np.save("/content/drive/MyDrive/aime/CNN/test_idx.txt",test_idx)

In [ ]:
import tensorflow as tf
import numpy as np
import os
from pathlib import Path
batch_size =  4
# Function to load and preprocess your data
def load_and_preprocess_data(file_path):
    # Load your data from file (e.g., images, CSV, etc.)
    #print(file_path)
    file_path = bytes.decode(file_path.numpy())
    data =  np.load(file_path)
    # Preprocess the data (normalization, resizing, etc.)
    data = data.transpose(3,0,1,2)
    # print(data.shape)
    #slice data
    #data = data[4:,:,:,:]
    data = tf.convert_to_tensor(data)
    data = tf.expand_dims(data, axis=-1)
    file_arr = file_path.split(".")
    file_path_y = file_arr[0]+"y."+ file_arr[1]

    y_res = int(np.load(str(file_path_y)))
    one_hot = np.zeros(10)
    one_hot[y_res] = 1
    return data, tf.convert_to_tensor(one_hot)

# Create a dataset from the list of file paths
file_paths = ["Brain_"+str(i)+".npy" for i in train_idx]
# train_dataset = tf.data.Dataset.list_files('/content/bird.jpg')
dataset = tf.data.Dataset.list_files(file_paths)

# Map the preprocessing function across dataset elements
dataset = dataset.map(lambda x: tf.py_function(load_and_preprocess_data, [x], [tf.float64,tf.float64]),
                      num_parallel_calls=tf.data.AUTOTUNE)

# Shuffle, repeat, and batch the dataset
dataset = dataset.shuffle(buffer_size=16).repeat().batch(batch_size)

# Prefetch to improve performance
dataset = dataset.prefetch(tf.data.AUTOTUNE)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Input, BatchNormalization, ReLU, TimeDistributed, LSTM

def create_4d_cnn_with_spacetime_kernel(input_shape, num_classes):
    # Input shape should be (batch_size, time_steps, height, width, channels)
    inputs = Input(shape=input_shape)

    # Example spacetime convolutional layer (extend kernel over two frames)
    # Kernel size here is (3, 3, 3), which extends over temporal and spatial dimensions
    conv_3d_lay1 = Conv3D(filters=32, kernel_size=(5, 5, 5), strides=(1, 1, 1), padding='same')
    x = TimeDistributed(conv_3d_lay1)(inputs)
    batch1 = BatchNormalization()
    x =  TimeDistributed(batch1)(x)
    x = ReLU()(x)
    max_pool_lay1 = MaxPooling3D(pool_size=(2, 2, 2), strides=(1, 1, 1))
    x = TimeDistributed(max_pool_lay1)(x)

    conv_3d_lay2 = Conv3D(filters=64, kernel_size=(4, 4, 4), strides=(1, 1, 1), padding='same')
    x = TimeDistributed(conv_3d_lay2)(x)
    batch2 = BatchNormalization()
    x =  TimeDistributed(batch2)(x)
    x = ReLU()(x)
    max_pool_lay2 = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2))
    x = TimeDistributed(max_pool_lay2)(x)

    conv_3d_lay3 = Conv3D(filters=128, kernel_size=(3, 3, 3), strides=(2,2 ,2), padding='same')
    x = TimeDistributed(conv_3d_lay3)(x)
    batch3 = BatchNormalization()
    x =  TimeDistributed(batch3)(x)
    x = ReLU()(x)
    max_pool_lay3 = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2))
    x = TimeDistributed(max_pool_lay3)(x)

    # conv_3d_lay4 = Conv3D(filters=128, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same')
    # x = TimeDistributed(conv_3d_lay4)(x)
    # batch4 = BatchNormalization()
    # x =  TimeDistributed(batch4)(x)
    # x = ReLU()(x)
    # max_pool_lay4 = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2))
    # x = TimeDistributed(max_pool_lay4)(x)
    x = TimeDistributed(Flatten())(x)
    x = TimeDistributed(Dense(32))(x)
    x = LSTM(64,return_sequences=False)(x)


    # Flatten and add Dense layers at the end
    # # Add more layers according to your specific problem, for example:
    # x = Conv3D(filters=64, kernel_size=(3, 3, 3), strides=(1, 2, 2), padding='same')(x)
    # x = BatchNormalization()(x)
    # x = ReLU()(x)
    # x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2))(x)

    # # Flatten and add Dense layers at the end
    # x = Flatten()(x)
    # x = Dense(512)(x)
    # x = ReLU()(x)
    x = Dense(num_classes, activation='softmax')(x)

    # # Create model
    model = Model(inputs=inputs, outputs=x)

    return model

# Example input shape: 10 frames (time_steps), each frame with size 64x64, and 3 channels
input_shape = (10,23,12,17,1)
num_classes = 10  # Example number of classes for classification

model = create_4d_cnn_with_spacetime_kernel(input_shape, num_classes)
model.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 10, 23, 12, 17, 1)   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_33                  │ (None, 10, 23, 12, 17, 32)  │           4,032 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_34                  │ (None, 10, 23, 12, 17, 32)  │             128 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_9 (ReLU)                       │ (None, 10, 23, 12, 17, 32)  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_35                  │ (None, 10, 22, 11, 16, 32)  │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_36                  │ (None, 10, 22, 11, 16, 64)  │         131,136 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_37                  │ (None, 10, 22, 11, 16, 64)  │             256 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_10 (ReLU)                      │ (None, 10, 22, 11, 16, 64)  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_38                  │ (None, 10, 11, 5, 8, 64)    │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_39                  │ (None, 10, 6, 3, 4, 128)    │         221,312 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_40                  │ (None, 10, 6, 3, 4, 128)    │             512 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_11 (ReLU)                      │ (None, 10, 6, 3, 4, 128)    │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_41                  │ (None, 10, 3, 1, 2, 128)    │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_42                  │ (None, 10, 768)             │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_43                  │ (None, 10, 32)              │          24,608 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (None, 64)                  │          24,8

 Total params: 407,466 (1.55 MB)

 Trainable params: 407,018 (1.55 MB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
from tensorflow.keras.metrics import CategoricalAccuracy
# Example input shape: 10 frames (time_steps), each frame with size 64x64, and 3 channels
# input_shape = (10,96,96,68)

num_classes = 10  # Example number of classes for classification

model = create_4d_cnn_with_spacetime_kernel(input_shape, num_classes)
optimizer = tf.optimizers.Adam(learning_rate=1e-4)
loss_object = tf.losses.CategoricalCrossentropy(from_logits=False)
val_acc_metric = CategoricalAccuracy()

In [ ]:
@tf.function
def train_step(inputs, targets):
    with tf.GradientTape() as tape:

        predictions = model(inputs, training=True)
        #print(predictions)
        #print(targets)
        loss = loss_object(targets, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [ ]:
@tf.function
def perform_validation(model, x, y):
    val_logits = model(x, training=False)
    return val_logits

In [ ]:
epochs = 100
batch_size = 4
steps_per_epoch = len(train_idx) // batch_size
val_best = 0

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    loss_tot = 0
    # Iterate over the batches of the dataset.
    for step, (inputs, targets) in enumerate(dataset.take(steps_per_epoch)):
        loss = train_step(inputs, targets)
        loss_tot += loss.numpy()
        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss_tot/(step+1)}")

    # End of epoch actions...
    # Perform validation
    for idx in val_idx:
        x_batch_val = np.load("Brain_"+str(idx)+".npy")
        x_batch_val = x_batch_val.transpose(3,0,1,2)
        x_batch_val = x_batch_val.reshape(1,*x_batch_val.shape)
        # x_batch_val = x_batch_val.reshape(1,96,96,68,10)
        #x_batch_val = x_batch_val[4:,:,:,:].reshape(1,6,96,96,68)
        x_batch_val = tf.convert_to_tensor(x_batch_val)
        x_batch_val = tf.expand_dims(x_batch_val, axis=-1)
        y_batch_val = np.load("Brain_"+str(idx)+"y.npy")
        y_batch_val = int(y_batch_val)
        one_hot = np.zeros(10)
        one_hot[y_batch_val] = 1
        y_batch_val = tf.convert_to_tensor(one_hot)
        val_logits = perform_validation(model, x_batch_val, y_batch_val)
        # Update validation metrics
        val_acc_metric.update_state(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    if(val_acc > val_best):
        val_best = val_acc
        model.save("model_alt.h5")
    val_acc_metric.reset_state()
    print("Training Loss: %.4f" % (float(loss_tot)/len(train_idx),))
    print("Validation acc: %.4f" % (float(val_acc),))

Epoch 1/100
Step 0, Loss: 1.7971081733703613
Step 100, Loss: 2.359218780357059


Training Loss: 0.5891
Validation acc: 0.0721
Epoch 2/100
Step 0, Loss: 2.3856453895568848
Step 100, Loss: 2.3152021724398772
Training Loss: 0.5804
Validation acc: 0.0631
Epoch 3/100
Step 0, Loss: 2.3253533840179443
Step 100, Loss: 2.3344125936527065


Training Loss: 0.5817
Validation acc: 0.0811
Epoch 4/100
Step 0, Loss: 2.3624629974365234
Step 100, Loss: 2.3171272419466833
Training Loss: 0.5794
Validation acc: 0.0811
Epoch 5/100
Step 0, Loss: 2.5155889987945557
Step 100, Loss: 2.3200543729385528


Training Loss: 0.5801
Validation acc: 0.0991
Epoch 6/100
Step 0, Loss: 2.4121034145355225
Step 100, Loss: 2.3163319106149203


Training Loss: 0.5805
Validation acc: 0.1081
Epoch 7/100
Step 0, Loss: 2.4181270599365234
Step 100, Loss: 2.3138176072942147
Training Loss: 0.5779
Validation acc: 0.0631
Epoch 8/100
Step 0, Loss: 2.4144182205200195
Step 100, Loss: 2.3073850504242546
Training Loss: 0.5766
Validation acc: 0.0721
Epoch 9/100
Step 0, Loss: 2.2260913848876953
Step 100, Loss: 2.3107853384301213
Training Loss: 0.5774
Validation acc: 0.0811
Epoch 10/100
Step 0, Loss: 2.2878470420837402
Step 100, Loss: 2.3062708330626536
Training Loss: 0.5768
Validation acc: 0.1081
Epoch 11/100
Step 0, Loss: 2.3249197006225586
Step 100, Loss: 2.3080386525333516
Training Loss: 0.5769
Validation acc: 0.0631
Epoch 12/100
Step 0, Loss: 2.2791013717651367
Step 100, Loss: 2.3050289555351333
Training Loss: 0.5763
Validation acc: 0.0811
Epoch 13/100
Step 0, Loss: 2.3258285522460938
Step 100, Loss: 2.306808497646067
Training Loss: 0.5767
Validation acc: 0.0901
Epoch 14/100
Step 0, Loss: 2.2753593921661377
Step 100, Loss: 2.3086126370005

Training Loss: 0.5754
Validation acc: 0.1171
Epoch 17/100
Step 0, Loss: 2.373727560043335
Step 100, Loss: 2.308694811150579
Training Loss: 0.5766
Validation acc: 0.0721
Epoch 18/100
Step 0, Loss: 2.3004884719848633
Step 100, Loss: 2.3051170807073613
Training Loss: 0.5763
Validation acc: 0.0721
Epoch 19/100
Step 0, Loss: 2.360264778137207
Step 100, Loss: 2.2997540270928107
Training Loss: 0.5761
Validation acc: 0.1081
Epoch 20/100
Step 0, Loss: 2.2573812007904053
Step 100, Loss: 2.302854960507686
Training Loss: 0.5760
Validation acc: 0.0721
Epoch 21/100
Step 0, Loss: 2.3428986072540283
Step 100, Loss: 2.302546201366009
Training Loss: 0.5752
Validation acc: 0.1081
Epoch 22/100
Step 0, Loss: 2.2233633995056152
Step 100, Loss: 2.306974758016001
Training Loss: 0.5769
Validation acc: 0.0721
Epoch 23/100
Step 0, Loss: 2.2860097885131836
Step 100, Loss: 2.3019448601373353
Training Loss: 0.5759
Validation acc: 0.0991
Epoch 24/100
Step 0, Loss: 2.396965742111206
Step 100, Loss: 2.306045976015601


KeyboardInterrupt: 

In [ ]:
# prompt: read in L_Heschl_gyrus_AAL_83.npy

import numpy as np

# Assuming the file is in the current working directory
brain_data_RH = np.load("R_Heschl_gyrus_AAL_84.npy")

# Now you can work with the loaded data, e.g., print its shape
print(brain_data_RH.shape)


(3, 611)


In [ ]:
# prompt: for each Brain_*.npy file zero out the voxels that does not belong in brain_data

import numpy as np

# Assuming brain_data is a boolean array indicating brain voxels
brain_mask = np.load("L_Heschl_gyrus_AAL_83.npy")
# brain_mask = np.load("/content/drive/MyDrive/aime/CNN/L_Heschl_gyrus_AAL_83.npy")


for j in range(num_samples):
  file_path = "Brain_"+str(j)+".npy"
  brain_data = np.load(file_path)
  new_brain_data = np.zeros(brain_data.shape)
  for i in range(brain_data_RH.shape[1]):
    new_brain_data[brain_data_RH[0,i]-60,brain_data_RH[1,i]-67,brain_data_RH[2,i]-23,:] = brain_data[brain_data_RH[0,i]-60,brain_data_RH[1,i]-67,brain_data_RH[2,i]-23,:]
  # Save the modified data back to the file
  np.save(file_path, new_brain_data)


In [ ]:
epochs = 100
batch_size = 4
steps_per_epoch = len(train_idx) // batch_size
val_best = 0

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    loss_tot = 0
    # Iterate over the batches of the dataset.
    for step, (inputs, targets) in enumerate(dataset.take(steps_per_epoch)):
        loss = train_step(inputs, targets)
        loss_tot += loss.numpy()
        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss_tot/(step+1)}")

    # End of epoch actions...
    # Perform validation
    for idx in val_idx:
        x_batch_val = np.load("Brain_"+str(idx)+".npy")
        x_batch_val = x_batch_val.transpose(3,0,1,2)
        x_batch_val = x_batch_val.reshape(1,*x_batch_val.shape)
        # x_batch_val = x_batch_val.reshape(1,96,96,68,10)
        #x_batch_val = x_batch_val[4:,:,:,:].reshape(1,6,96,96,68)
        x_batch_val = tf.convert_to_tensor(x_batch_val)
        x_batch_val = tf.expand_dims(x_batch_val, axis=-1)
        y_batch_val = np.load("Brain_"+str(idx)+"y.npy")
        y_batch_val = int(y_batch_val)
        one_hot = np.zeros(10)
        one_hot[y_batch_val] = 1
        y_batch_val = tf.convert_to_tensor(one_hot)
        val_logits = perform_validation(model, x_batch_val, y_batch_val)
        # Update validation metrics
        val_acc_metric.update_state(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    if(val_acc > val_best):
        val_best = val_acc
        model.save("model_alt.h5")
    val_acc_metric.reset_state()
    print("Training Loss: %.4f" % (float(loss_tot)/len(train_idx),))
    print("Validation acc: %.4f" % (float(val_acc),))

Epoch 1/100
Step 0, Loss: 2.325343608856201
Step 100, Loss: 2.3137859353924743


Training Loss: 0.5783
Validation acc: 0.1081
Epoch 2/100
Step 0, Loss: 2.3379697799682617
Step 100, Loss: 2.3053948407125944
Training Loss: 0.5770
Validation acc: 0.0811
Epoch 3/100
Step 0, Loss: 2.3234245777130127
Step 100, Loss: 2.3057427807609634
Training Loss: 0.5766
Validation acc: 0.0901
Epoch 4/100
Step 0, Loss: 2.311617374420166
Step 100, Loss: 2.3032739492926266
Training Loss: 0.5763
Validation acc: 0.0901
Epoch 5/100
Step 0, Loss: 2.2883594036102295
Step 100, Loss: 2.3017203241291613
Training Loss: 0.5758
Validation acc: 0.0811
Epoch 6/100
Step 0, Loss: 2.288486957550049
Step 100, Loss: 2.3011993252404848
Training Loss: 0.5758
Validation acc: 0.0901
Epoch 7/100
Step 0, Loss: 2.3000266551971436
Step 100, Loss: 2.303874882140962
Training Loss: 0.5760
Validation acc: 0.0631
Epoch 8/100
Step 0, Loss: 2.2854132652282715
Step 100, Loss: 2.3036872254739893
Training Loss: 0.5756
Validation acc: 0.0901
Epoch 9/100
Step 0, Loss: 2.347957134246826
Step 100, Loss: 2.3021180157614225
Trai

Training Loss: 0.5760
Validation acc: 0.1261
Epoch 30/100
Step 0, Loss: 2.297624349594116
Step 100, Loss: 2.3062448171105716
Training Loss: 0.5764
Validation acc: 0.0721
Epoch 31/100
Step 0, Loss: 2.2099359035491943
Step 100, Loss: 2.29549237997225
Training Loss: 0.5754
Validation acc: 0.0901
Epoch 32/100
Step 0, Loss: 2.3396964073181152
Step 100, Loss: 2.299308181989311
Training Loss: 0.5758
Validation acc: 0.0721
Epoch 33/100
Step 0, Loss: 2.250347375869751
Step 100, Loss: 2.2918919525524175
Training Loss: 0.5727
Validation acc: 0.0811
Epoch 34/100
Step 0, Loss: 2.3485493659973145
Step 100, Loss: 2.2930456246479904


Training Loss: 0.5729
Validation acc: 0.1441
Epoch 35/100
Step 0, Loss: 2.384152412414551
Step 100, Loss: 2.295515263434684
Training Loss: 0.5746
Validation acc: 0.1261
Epoch 36/100
Step 0, Loss: 2.25662899017334
Step 100, Loss: 2.2942593569802767
Training Loss: 0.5719
Validation acc: 0.0901
Epoch 37/100
Step 0, Loss: 2.3034162521362305
Step 100, Loss: 2.2704450588415166
Training Loss: 0.5701
Validation acc: 0.0811
Epoch 38/100
Step 0, Loss: 2.2007243633270264
Step 100, Loss: 2.278236103529977
Training Loss: 0.5691
Validation acc: 0.1171
Epoch 39/100
Step 0, Loss: 2.204221725463867
Step 100, Loss: 2.2794576682666743
Training Loss: 0.5702
Validation acc: 0.0991
Epoch 40/100
Step 0, Loss: 2.2328436374664307
Step 100, Loss: 2.250506574564641
Training Loss: 0.5633
Validation acc: 0.0991
Epoch 41/100
Step 0, Loss: 2.3126468658447266
Step 100, Loss: 2.248175105245987
Training Loss: 0.5635
Validation acc: 0.1441
Epoch 42/100
Step 0, Loss: 1.9069342613220215
Step 100, Loss: 2.255868471494996
T

Training Loss: 0.5485
Validation acc: 0.1712
Epoch 50/100
Step 0, Loss: 2.2161927223205566
Step 100, Loss: 2.1827642846815656
Training Loss: 0.5439
Validation acc: 0.0991
Epoch 51/100
Step 0, Loss: 2.355868101119995
Step 100, Loss: 2.156328030151896
Training Loss: 0.5391
Validation acc: 0.1171
Epoch 52/100
Step 0, Loss: 2.023317575454712
Step 100, Loss: 2.1339491806407964
Training Loss: 0.5366
Validation acc: 0.1261
Epoch 53/100
Step 0, Loss: 2.00838303565979
Step 100, Loss: 2.125826631442155
Training Loss: 0.5351
Validation acc: 0.1261
Epoch 54/100
Step 0, Loss: 2.126699924468994
Step 100, Loss: 2.110076603322926
Training Loss: 0.5297
Validation acc: 0.1171
Epoch 55/100
Step 0, Loss: 2.445472002029419
Step 100, Loss: 2.1372838693090004
Training Loss: 0.5356
Validation acc: 0.1081
Epoch 56/100
Step 0, Loss: 2.65710711479187
Step 100, Loss: 2.1238521384720754
Training Loss: 0.5362
Validation acc: 0.1622
Epoch 57/100
Step 0, Loss: 1.9103951454162598
Step 100, Loss: 2.101788699036778
Trai

Training Loss: 0.5194
Validation acc: 0.2072
Epoch 61/100
Step 0, Loss: 2.1290831565856934
Step 100, Loss: 2.0764232956536928
Training Loss: 0.5212
Validation acc: 0.1982
Epoch 62/100
Step 0, Loss: 2.0420596599578857
Step 100, Loss: 2.0382293344724296
Training Loss: 0.5132
Validation acc: 0.1622
Epoch 63/100
Step 0, Loss: 2.166501998901367
Step 100, Loss: 2.0541510888845615
Training Loss: 0.5152
Validation acc: 0.1351
Epoch 64/100
Step 0, Loss: 1.4346373081207275
Step 100, Loss: 1.987649928225149
Training Loss: 0.5016
Validation acc: 0.1622
Epoch 65/100
Step 0, Loss: 1.7372350692749023
Step 100, Loss: 2.0042912570556792
Training Loss: 0.4985
Validation acc: 0.1892
Epoch 66/100
Step 0, Loss: 1.9920457601547241
Step 100, Loss: 1.9639292506888362
Training Loss: 0.4950
Validation acc: 0.1802
Epoch 67/100
Step 0, Loss: 1.7848058938980103
Step 100, Loss: 2.022849644764815
Training Loss: 0.4995
Validation acc: 0.1892
Epoch 68/100
Step 0, Loss: 2.0676355361938477
Step 100, Loss: 1.960047328826

Training Loss: 0.4520
Validation acc: 0.2252
Epoch 82/100
Step 0, Loss: 1.8859370946884155
Step 100, Loss: 1.7879321563361894
Training Loss: 0.4401
Validation acc: 0.2072
Epoch 83/100
Step 0, Loss: 1.6651077270507812
Step 100, Loss: 1.7119918766588267
Training Loss: 0.4334
Validation acc: 0.2072
Epoch 84/100
Step 0, Loss: 1.2072865962982178
Step 100, Loss: 1.76102212100926
Training Loss: 0.4445
Validation acc: 0.1982
Epoch 85/100
Step 0, Loss: 2.128755807876587
Step 100, Loss: 1.7249144044252906
Training Loss: 0.4296
Validation acc: 0.1441
Epoch 86/100
Step 0, Loss: 1.775880217552185
Step 100, Loss: 1.7186187865710494
Training Loss: 0.4316
Validation acc: 0.1892
Epoch 87/100
Step 0, Loss: 1.057159662246704
Step 100, Loss: 1.709537644197445
Training Loss: 0.4197
Validation acc: 0.1982
Epoch 88/100
Step 0, Loss: 1.56492018699646
Step 100, Loss: 1.6308653047769377
Training Loss: 0.4146
Validation acc: 0.2072
Epoch 89/100
Step 0, Loss: 1.6814519166946411
Step 100, Loss: 1.6294129732811804


In [ ]:
# prompt: Build a simple LSTM model that takes in (594,10) input and output a 10 class classification for 10 temporal steps

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Build the LSTM model
model_r = Sequential()
model_r.add(LSTM(units=64, return_sequences=True, input_shape=(10, 611)))  # Adjust units as needed
model_r.add(LSTM(units=32, return_sequences=False))
model_r.add(Dense(units=10, activation='softmax'))

# Compile the model
model_r.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model_r.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_9 (LSTM)                        │ (None, 10, 64)              │         173,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_10 (LSTM)                       │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 10)                  │             330 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 185,802 (725.79 KB)

 Trainable params: 185,802 (725.79 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
X_LSTM_R = np.zeros((738,10,611))
Y_LSTM = np.zeros(738)
for i in range(738):
  meta_data = np.load("Brain_"+str(i)+".npy")
  meta_data_y = np.load("Brain_"+str(i)+"y.npy")
  Y_LSTM[i] = meta_data_y
  for j in range(brain_data_RH.shape[1]):
    X_LSTM_R[i,:,j] = meta_data[brain_data_RH[0,j]-60,brain_data_RH[1,j]-67,brain_data_RH[2,j]-23,:]



In [ ]:
Y_LSTM

array([8., 2., 4., 3., 5., 0., 7., 1., 6., 9., 8., 2., 4., 3., 5., 0., 7.,
       1., 6., 9., 8., 2., 4., 3., 5., 0., 7., 1., 6., 9., 8., 2., 4., 3.,
       5., 0., 7., 1., 6., 9., 8., 0., 7., 2., 6., 8., 4., 9., 3., 1., 5.,
       0., 7., 2., 6., 8., 4., 9., 3., 1., 5., 0., 7., 2., 6., 8., 4., 9.,
       3., 1., 5., 0., 7., 2., 6., 8., 4., 9., 3., 1., 5., 0., 9., 6., 3.,
       2., 4., 8., 5., 1., 0., 7., 9., 6., 3., 2., 4., 8., 5., 1., 0., 7.,
       9., 6., 3., 2., 4., 8., 5., 1., 0., 7., 9., 6., 3., 2., 4., 8., 5.,
       1., 0., 7., 9., 9., 0., 4., 2., 7., 3., 6., 5., 1., 8., 9., 0., 4.,
       2., 7., 3., 6., 5., 1., 8., 9., 0., 4., 2., 7., 3., 6., 5., 1., 8.,
       9., 0., 4., 2., 7., 3., 6., 5., 1., 8., 9., 7., 8., 4., 1., 0., 5.,
       6., 2., 3., 9., 7., 8., 4., 1., 0., 5., 6., 2., 3., 9., 7., 8., 4.,
       1., 0., 5., 6., 2., 3., 9., 7., 8., 4., 1., 0., 5., 6., 2., 3., 9.,
       7., 7., 3., 8., 1., 5., 0., 9., 6., 4., 2., 7., 3., 8., 1., 5., 0.,
       9., 6., 4., 2., 7.

In [ ]:
# prompt: generate the training loop for X_LSTM_R and Y_LSTM for model_r

from tensorflow.keras.utils import to_categorical

# Convert Y_LSTM to one-hot encoding
Y_LSTM_cat = to_categorical(Y_LSTM, num_classes=10)

# Split data into training, validation, and test sets
X_train_r = X_LSTM_R[train_idx]
y_train_r = Y_LSTM_cat[train_idx]
X_val_r = X_LSTM_R[val_idx]
y_val_r = Y_LSTM_cat[val_idx]
X_test_r = X_LSTM_R[test_idx]
y_test_r = Y_LSTM_cat[test_idx]

# Train the LSTM model
epochs = 100
batch_size = 4
history = model_r.fit(
    X_train_r, y_train_r,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_val_r, y_val_r)
)


Epoch 1/100
129/129 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.1089 - loss: 2.3500 - val_accuracy: 0.0721 - val_loss: 2.3042
Epoch 2/100
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0906 - loss: 2.3204 - val_accuracy: 0.0811 - val_loss: 2.3251
Epoch 3/100
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0868 - loss: 2.3035 - val_accuracy: 0.1081 - val_loss: 2.3034
Epoch 4/100
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0739 - loss: 2.3123 - val_accuracy: 0.1081 - val_loss: 2.3115
Epoch 5/100
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0836 - loss: 2.3128 - val_accuracy: 0.1081 - val_loss: 2.3078
Epoch 6/100
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.1254 - loss: 2.3066 - val_accuracy: 0.0631 - val_loss: 2.3244
Epoch 7/100
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.1081 - loss: 2.3026 - val_accuracy: 0.0991 - val_loss: 2.3138
Epoch 8/100
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.1052 - loss: 2.3040 - val_accu